# Local Delivery Planner for Amazon Fresh Orders

A sample notebook for generating local delivery plans of Amazon Fresh Orders

### Imports

In [ ]:
import json

import numpy as np
import pandas as pd

from amznoptim.solver.GurobiMIP import DepotVRPGurobiFresh
from amznoptim.utils import preprocess

### Input preparation

In [ ]:
# Helper function to regenerate delivery windows for demo
def regenerate_delivery_windows(orders):
    rng = np.random.default_rng()
    now = pd.Timestamp.now()
    start_time_max = now + pd.Timedelta(hours=4)
    start_time_min = now + pd.Timedelta(hours=0.2)
    for order in orders:
        start_timestamp = rng.integers(
            start_time_min.value, start_time_max.value
        )
        start_timestamp_pd = pd.Timestamp(start_timestamp)
        end_timestamp_pd = start_timestamp_pd + pd.Timedelta(
            hours=rng.integers(1, 3)
        )
        order["delivery_window_start"] = start_timestamp_pd.value
        order["delivery_window_end"] = end_timestamp_pd.value

    return orders



In [ ]:
# Example order csv file
order_csv = "input_data/fresh/orders.csv"
pd.read_csv(order_csv)

In [ ]:
# Example depot info file
depot_json = "input_data/depot.json"
with open(depot_json, "r") as f:
    depot_info = json.load(f)
depot_info

In [ ]:
# Example vehicle config json
vehicle_json = "config/vehicle_config.json"
# Example Google Map RouteMatrix json
route_matrix_json = "input_data/fresh/gmap_matrix.json"
# Example Google Map Address Validation json
address_validation_json = "input_data/fresh/gmap_validation.json"

### Solver data processing

In [ ]:
# Preprocess the order data csv and extract order and address info
order_info, address_info = preprocess.sameday_order_info_from_csv(order_csv)

# Regenerate delivery windows for orders
order_info = regenerate_delivery_windows(order_info)
order_info

### Use Gurobi MIP Solver (May require Gurobi WLS License File)

For licensing, please visit https://www.gurobi.com/features/web-license-service/

Alternatively, you can run Google OR-Tools CP-SAT solver below (slower than Gurobi)

In [ ]:
# Define the Gurobi solver and process the data
solver = DepotVRPGurobiFresh(
    depot_data=depot_info,
    order_data=order_info,
    address_data=address_info,
)

solver.process_data(
    vehicle_data_path=vehicle_json,
    route_matrix_json=route_matrix_json,
)

solver.set_stopping_time(
    address_validation_json=address_validation_json
)

In [ ]:
# Optional: use your own Gurobi WSL license file
# solver.use_licensed_environment('path/to/your/gurobi.lic')

# The fresh order delivery optimization problem has larger constraint space,
# so it may take longer to solve than the standard depot VRP.
# Optional: you can set the solver runtime limit to get a feasible solution
# solver.set_runtime_limit(300) # seconds

#### Run the solver

In [ ]:
result = solver.solve()
result

#### Process the result to human-readable format

In [ ]:
plan_readable = solver.generate_plan(result, save_path=None)
plan_readable